In [11]:
import os, json
import kfp
from dkube.sdk import *
from dkube.pipelines import dkube_training_op

In [12]:
components_url = "/mnt/dkube/pipeline/components/"
dkube_training_op = kfp.components.load_component_from_file(components_url + "training/component.yaml")
dkube_serving_op = kfp.components.load_component_from_file(components_url + "serving/component.yaml")
token = os.getenv("DKUBE_USER_ACCESS_TOKEN")
client = kfp.Client(host=os.getenv("KF_PIPELINES_ENDPOINT"), existing_token=token, namespace=os.getenv("USERNAME"))

In [ ]:
@kfp.dsl.pipeline(
    name='dkube-titanic-pl',
    description='sample titanic pipeline with dkube components'
)
def titanic_pipeline():
    train       = dkube_training_op(auth_token = token,container = '{"image":"docker.io/ocdr/dkube-datascience-tf-cpu:v2.0.0-3"}',
                                    framework="tensorflow", version="2.0.0",
                                    program="titanic", run_script="python titanic/training.py",
                                    datasets='["titanic"]', outputs='["titanic"]',
                                    input_dataset_mounts='["/data"]',
                                    output_mounts='["/model"]',
                                    envs='[{"EPOCHS": "1"}]')
    
    serving     = dkube_serving_op(auth_token = token,model = train.outputs['artifact'], device='cpu', 
                                    serving_image='{"image":"ocdr/sklearnserver:0.23.2"}',
                                    transformer_image='{"image":"ocdr/dkube-datascience-tf-cpu:v2.0.0"}',
                                    transformer_project="titanic",
                                    transformer_code='titanic/transformer.py').after(train)

In [ ]:
client.create_run_from_pipeline_func(titanic_pipeline, arguments={})